### Training the Detection and Attribution Network:

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
import os
import torch

# Load the pre-trained Xception model
base_model = tf.keras.applications.Xception(input_shape=(256, 256, 3),
                                            include_top=False,
                                            weights='imagenet')

# Unfreeze the base model
base_model.trainable = True

# Fine-tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

# Defining my ArcFace Function
class ArcFace(layers.Layer):
    def __init__(self, n_classes=5, s=64.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizer

    def build(self, input_shape):
        super(ArcFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                 shape=(input_shape[0][-1], self.n_classes),
                                 initializer='glorot_uniform',
                                 trainable=True,
                                 regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = tf.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        theta = tf.acos(tf.clip_by_value(logits, -1.0 + tf.keras.backend.epsilon(), 1.0 - tf.keras.backend.epsilon()))
        target_logits = tf.cos(theta + self.m)
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'regularizer': self.regularizer,
        })
        return config

# Add new layers
x = base_model.output
x = layers.Flatten()(x)
x = layers.Dense(1024, activation='relu')(x)
x = layers.Dropout(0.5)(x)

# Placeholder for label input
y = tf.keras.Input(shape=(5,))  # We have 5 classes currerntly
output = ArcFace(n_classes=5)([x, y])
print(output)

# Define the new model
model = Model(inputs=[base_model.input, y], outputs=output)

# Compile the model
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.00001),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy'])

# Define the parameters for the ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

# Load the training data
train_data = datagen.flow_from_directory(
    'multi-detector/train',
    target_size=(256, 256),
    batch_size=16,
    class_mode='sparse')

# Load the validation data
validation_data = datagen.flow_from_directory(
    'multi-detector/validation',
    target_size=(256, 256),
    batch_size=16,
    class_mode='sparse')

# Load the test data
test_data = datagen.flow_from_directory(
    'multi-detector/test',
    target_size=(256, 256),
    batch_size=16,
    class_mode='sparse')

def arcface_generator(generator):
    while True:
        data = next(generator)
        yield [data[0], tf.one_hot(data[1], depth=5)], tf.one_hot(data[1], depth=5)

# Create the new generators
train_generator = arcface_generator(train_data)
validation_generator = arcface_generator(validation_data)
test_generator = arcface_generator(test_data)

# Set up callbacks
early_stop = EarlyStopping(monitor='val_loss', patience=20)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=5, min_lr=0.00001)
os.makedirs('trained_models', exist_ok=True)
model_checkpoint = ModelCheckpoint('trained_models/Xception_Multi_Detector.h5', save_best_only=True)

callbacks = [early_stop, reduce_lr, model_checkpoint]

# Fit the model on the batches generated by datagen.flow()
model.fit(train_generator,
          validation_data=validation_generator,
          steps_per_epoch=len(train_data), 
          validation_steps=len(validation_data),
          epochs=100,
          callbacks=callbacks)

KerasTensor(type_spec=TensorSpec(shape=(None, 5), dtype=tf.float32, name=None), name='arc_face/Softmax:0', description="created by layer 'arc_face'")
Found 65988 images belonging to 5 classes.
Found 21073 images belonging to 5 classes.
Found 21073 images belonging to 5 classes.
Epoch 1/100
4125/4125 [==============================] - 1127s 272ms/step - loss: 18.8503 - accuracy: 0.2754 - val_loss: 15.0717 - val_accuracy: 0.6642 - lr: 1.0000e-05
Epoch 2/100
4125/4125 [==============================] - 319s 77ms/step - loss: 13.7258 - accuracy: 0.5192 - val_loss: 10.9662 - val_accuracy: 0.7195 - lr: 1.0000e-05
Epoch 3/100
4125/4125 [==============================] - 307s 74ms/step - loss: 11.1549 - accuracy: 0.5861 - val_loss: 8.1308 - val_accuracy: 0.7806 - lr: 1.0000e-05
Epoch 4/100
4125/4125 [==============================] - 304s 74ms/step - loss: 8.3172 - accuracy: 0.6548 - val_loss: 4.8717 - val_accuracy: 0.8681 - lr: 1.0000e-05
Epoch 5/100
4125/4125 [==============================]

### Evaluate the Network:

In [4]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import load_model
import os
import random

# Defining my ArcFace Function
class ArcFace(layers.Layer):
    def __init__(self, n_classes=5, s=64.0, m=0.50, regularizer=None, **kwargs):
        super(ArcFace, self).__init__(**kwargs)
        self.n_classes = n_classes
        self.s = s
        self.m = m
        self.regularizer = regularizer

    def build(self, input_shape):
        super(ArcFace, self).build(input_shape[0])
        self.W = self.add_weight(name='W',
                                 shape=(input_shape[0][-1], self.n_classes),
                                 initializer='glorot_uniform',
                                 trainable=True,
                                 regularizer=self.regularizer)

    def call(self, inputs):
        x, y = inputs
        c = tf.shape(x)[-1]
        # normalize feature
        x = tf.nn.l2_normalize(x, axis=1)
        # normalize weights
        W = tf.nn.l2_normalize(self.W, axis=0)
        # dot product
        logits = x @ W
        # add margin
        theta = tf.acos(tf.clip_by_value(logits, -1.0 + tf.keras.backend.epsilon(), 1.0 - tf.keras.backend.epsilon()))
        target_logits = tf.cos(theta + self.m)
        logits = logits * (1 - y) + target_logits * y
        # feature re-scale
        logits *= self.s
        out = tf.nn.softmax(logits)

        return out

    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'n_classes': self.n_classes,
            's': self.s,
            'm': self.m,
            'regularizer': self.regularizer,
        })
        return config
    
# Load the saved model
# model = load_model('trained_models/Xception_Multi_Detector.h5', custom_objects={'ArcFace': ArcFace})

# Define the parameters for the ImageDataGenerator
datagen = ImageDataGenerator(rescale=1./255)

# Load the training data
train_data = datagen.flow_from_directory(
    'multi-detector/train',
    target_size=(256, 256),
    batch_size=16,
    class_mode='sparse')

# Load the validation data
validation_data = datagen.flow_from_directory(
    'multi-detector/validation',
    target_size=(256, 256),
    batch_size=16,
    class_mode='sparse')

# Load the test data
test_data = datagen.flow_from_directory(
    'multi-detector/test',
    target_size=(256, 256),
    batch_size=16,
    class_mode='sparse')

def arcface_generator(generator):
    while True:
        data = next(generator)
        yield [data[0], tf.one_hot(data[1], depth=5)], tf.one_hot(data[1], depth=5)

# Create the new generators
train_generator = arcface_generator(train_data)
validation_generator = arcface_generator(validation_data)
test_generator = arcface_generator(test_data)

# Evaluate the model on the test data
test_loss, test_acc = model.evaluate(test_generator, steps=10000)
print(f'Test accuracy: {test_acc}')

Found 65988 images belonging to 5 classes.
Found 21073 images belonging to 5 classes.
Found 21073 images belonging to 5 classes.
10000/10000 [==============================] - 718s 72ms/step - loss: 0.2807 - accuracy: 0.9946
Test accuracy: 0.9946339726448059


### Compute individual Predictions:

In [6]:
from tensorflow.keras.preprocessing import image

# Load the image
img_path = 'multi-detector/test/No Attack/1141/1141_l_1.png'
img = image.load_img(img_path, target_size=(256, 256))

# Convert the image to a numpy array
x = image.img_to_array(img)

# Scale the image
x = x / 255.0

# Expand the dimensions to match the shape that the model expects
x = np.expand_dims(x, axis=0)

# Prepare the dummy label input for the ArcFace layer
dummy_label_input = np.zeros((1, 5))

# Make the prediction
predictions = model.predict([x, dummy_label_input])

# The predictions are probabilities for each class. You can get the class with the highest probability like this:
predicted_class = np.argmax(predictions[0])

print(f"The predicted class is: {predicted_class}")

# Get the class indices
class_indices = train_data.class_indices

# Get the class names
class_names = list(class_indices.keys())

# Print the predicted class name
print(f"The predicted class is: {class_names[3]}")

1/1 [==============================] - 0s 34ms/step
The predicted class is: 3
The predicted class is: No Attack


In [7]:
from tensorflow.keras.preprocessing import image

# Load the image
img_path = 'multi-detector/test/PGD/1141/1141_l_1_pgd_01.png'
img = image.load_img(img_path, target_size=(256, 256))

# Convert the image to a numpy array
x = image.img_to_array(img)

# Scale the image
x = x / 255.0

# Expand the dimensions to match the shape that the model expects
x = np.expand_dims(x, axis=0)

# Prepare the dummy label input for the ArcFace layer
dummy_label_input = np.zeros((1, 5))

# Make the prediction
predictions = model.predict([x, dummy_label_input])

# The predictions are probabilities for each class. You can get the class with the highest probability like this:
predicted_class = np.argmax(predictions[0])

print(f"The predicted class is: {predicted_class}")

# Get the class indices
class_indices = train_data.class_indices

# Get the class names
class_names = list(class_indices.keys())

# Print the predicted class name
print(f"The predicted class is: {class_names[4]}")

1/1 [==============================] - 0s 34ms/step
The predicted class is: 4
The predicted class is: PGD


In [9]:
from tensorflow.keras.preprocessing import image

# Load the image
img_path = 'multi-detector/test/MIM/3265/3265_l_1_mim_001.png'
img = image.load_img(img_path, target_size=(256, 256))

# Convert the image to a numpy array
x = image.img_to_array(img)

# Scale the image
x = x / 255.0

# Expand the dimensions to match the shape that the model expects
x = np.expand_dims(x, axis=0)

# Prepare the dummy label input for the ArcFace layer
dummy_label_input = np.zeros((1, 5))

# Make the prediction
predictions = model.predict([x, dummy_label_input])

# The predictions are probabilities for each class. You can get the class with the highest probability like this:
predicted_class = np.argmax(predictions[0])

print(f"The predicted class is: {predicted_class}")

# Get the class indices
class_indices = train_data.class_indices

# Get the class names
class_names = list(class_indices.keys())

# Print the predicted class name
print(f"The predicted class is: {class_names[predicted_class]}")

1/1 [==============================] - 0s 27ms/step
The predicted class is: 2
The predicted class is: MIM
